### Compute the month-on-month growth rate in sales for each product, highlighting products with more than 10% growth for consecutive months.

**Solving in both SQL & Python**

In [3]:
# %pip install pyodbc
# import pyodbc

In [4]:
import pandas as pd
import numpy as np
import pyodbc
from warnings import filterwarnings

filterwarnings("ignore", category=UserWarning, message='.*pandas only supports SQLAlchemy connectable.*')


https://4js.com/online_documentation/fjs-fgl-3.00.05-manual-html/c_fgl_DataConversions_format_datetimes.html

In [5]:
df2 = pd.DataFrame({
    'Month': ['Jan', 'Feb', 'Mar', 'Jan', 'Feb', 'Mar'],
    'Product': ['A', 'A', 'A', 'B', 'B', 'B'],
    'Sales': [200, 220, 240, 150, 165, 180]
})
print(df2)


  Month Product  Sales
0   Jan       A    200
1   Feb       A    220
2   Mar       A    240
3   Jan       B    150
4   Feb       B    165
5   Mar       B    180


In [6]:
# Map month names to numerical values
month_mapping = {'Jan': 1, 'Feb': 2, 'Mar': 3}
df2['MonthNum'] = df2['Month'].map(month_mapping)

df2 = df2.sort_values(by=['Product', 'MonthNum'])

In [7]:
# Compute the prev month's sales using the shift function
df2['PrevMonthSales'] = df2.groupby('Product')['Sales'].shift(1)
df2['GrowthRate M1'] = (df2['Sales'] - df2['PrevMonthSales']) *100/ df2['PrevMonthSales']

# Filter out rows where NextMonthSales is NaN 
#result = df2.dropna(subset=['NextMonthSales'])



# ANOTHER WAY

df2['GrowthRate M2'] = df2.groupby('Product')['Sales'].pct_change() * 100
print(df2)

  Month Product  Sales  MonthNum  PrevMonthSales  GrowthRate M1  GrowthRate M2
0   Jan       A    200         1             NaN            NaN            NaN
1   Feb       A    220         2           200.0      10.000000      10.000000
2   Mar       A    240         3           220.0       9.090909       9.090909
3   Jan       B    150         1             NaN            NaN            NaN
4   Feb       B    165         2           150.0      10.000000      10.000000
5   Mar       B    180         3           165.0       9.090909       9.090909


### Solving in SQL

#### Creating Connection and tables

In [9]:
# Define the connection string
conn_str = (
    r'DRIVER={SQL Server Native Client 11.0};'
    r'SERVER=server_name;'
    r'DATABASE=AdvanceSQL;'
    r'Trusted_Connection=yes;'
)

# Establish the connection
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()


In [11]:
# Define the SQL command to create a table
create_table_sql = """
CREATE TABLE MOM (
    Month date,
    Product VARCHAR(10),
    Sales DECIMAL(10, 2)
)
"""

# Execute the SQL command
cursor.execute(create_table_sql)
conn.commit()
print("Table created successfully.")


# insert query
# insert_sql = """
# INSERT INTO MOM (Month, Product, Sales)
# VALUES (?, ?, ?)
# """
# Example data to insert
# data = [
#     ('Jan', 'A', 200),
#     ('Feb', 'A', 220),
#     ('Mar', 'A', 240),
#     ('Jan', 'B', 150),
#     ('Feb', 'B', 165),
#     ('Mar', 'B', 180)
# ]
# cursor.executemany(insert_sql,data)


insert_sql = """
INSERT INTO MOM (Month, Product, Sales)
VALUES 
    (CAST('2024-01-01' AS DATE), 'A', 200), -- Jan
    (CAST('2024-02-01' AS DATE), 'A', 220), -- Feb
    (CAST('2024-03-01' AS DATE), 'A', 240), -- Mar
    (CAST('2024-01-01' AS DATE), 'B', 150), -- Jan
    (CAST('2024-02-01' AS DATE), 'B', 165), -- Feb
    (CAST('2024-03-01' AS DATE), 'B', 180); -- Mar
"""

# Execute the SQL command for each record
cursor.execute(insert_sql)
conn.commit()
print("Records inserted successfully.")


In [12]:
query = """ 
WITH SalesWithLead AS (
    SELECT
        Month,
        Product,
        Sales,
        lag(Sales) OVER (PARTITION BY Product ORDER BY Month) AS PrevMonthSales
    FROM
        MOM
)
SELECT
    Month,
    Product,
    Sales,
    PrevMonthSales,
    (Sales - PrevMonthSales) * 100.0 / PrevMonthSales AS GrowthRate
FROM
    SalesWithLead;
"""

In [13]:
# Execute the query and fetch results into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the result
print(df)

        Month Product  Sales  PrevMonthSales  GrowthRate
0  2024-01-01       A  200.0             NaN         NaN
1  2024-02-01       A  220.0           200.0   10.000000
2  2024-03-01       A  240.0           220.0    9.090909
3  2024-01-01       B  150.0             NaN         NaN
4  2024-02-01       B  165.0           150.0   10.000000
5  2024-03-01       B  180.0           165.0    9.090909


g:\Downloads\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [14]:
# close the cursor and connection
cursor.close()
conn.close()